# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os

import sys
sys.path.insert(1, './../src/')

import time 

import load_option_data_01 

from pathlib import Path

import bsm_pricer as bsm
import config
import datetime
import level_1_filters as f1
import level_2_filters as f2
import level_3_filters as f3
import load_option_data_01 
import load_option_data_01 as l1


import time 
import warnings
import wrds

from scipy.stats import norm
from scipy.spatial.distance import cdist

In [2]:
import importlib

In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
warnings.filterwarnings("ignore")

In [4]:
OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME

START_DATE_01 =config.START_DATE_01
END_DATE_01 = config.END_DATE_01

START_DATE_02 =config.START_DATE_02
END_DATE_02 = config.END_DATE_02


In [5]:
DATE_RANGE =f'{pd.Timestamp(START_DATE_01):%Y-%m}_{pd.Timestamp(END_DATE_02):%Y-%m}'

# Functions

In [6]:
# --- Black-Scholes elasticity ---
def bs_elasticity(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T))
        delta = norm.cdf(d1)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)) - S * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
    return (delta * S / price), price

# Gaussian kernel function
def kernel_weights(m_grid, ttm_grid, k_s, ttm, bw_m=0.0125, bw_t=10):
    m_grid = np.asarray(m_grid, dtype=float)
    ttm_grid = np.asarray(ttm_grid, dtype=float)
    x = (m_grid - k_s) / bw_m
    y = (ttm_grid - ttm) / bw_t
    dist_sq = x**2 + y**2
    weights = np.exp(-0.5 * dist_sq)
    return weights / weights.sum() if weights.sum() > 0 else np.zeros_like(weights)

# --- Construct a single day portfolio ---
def construct_portfolio(data, k_s_target, ttm_target, option_type='call', r=0.01):
    subset = data[(data['option_type'] == option_type)]
    weights = kernel_weights(subset['moneyness'], subset['ttm'], k_s_target, ttm_target)
    subset = subset.assign(weight=weights)
    subset = subset[subset['weight'] > 0.01]
    subset['weight'] /= subset['weight'].sum()

    # Leverage-adjusted returns
    elast, price = bs_elasticity(
        S=subset['underlying'], K=subset['strike'], T=subset['ttm']/365,
        r=r, sigma=subset['iv'], option_type=option_type
    )
    subset['leverage_return'] = subset['daily_return'] / elast

    return (subset['leverage_return'] * subset['weight']).sum()

# --- Main Loop (simplified) ---
def build_portfolios(option_data, m_grid, ttm_grid, option_types=['call', 'put']):
    portfolios = []
    for opt_type in option_types:
        for k_s in m_grid:
            for ttm in ttm_grid:
                ret = construct_portfolio(option_data, k_s, ttm, option_type=opt_type)
                portfolios.append({
                    'type': opt_type,
                    'moneyness': k_s,
                    'ttm': ttm,
                    'return': ret
                })
    return pd.DataFrame(portfolios)

def calc_kernel_weights(spx_mod):
    """ Calculate kernel weights for each option in the SPX dataset based on moneyness and maturity targets.
    This function iterates through predefined moneyness and maturity targets, applies kernel weights to candidate options."""
    
    # Define moneyness and maturity targets from the paper
    moneyness_targets = [0.925, 0.950, 0.975, 1.000, 1.025, 1.050, 1.075, 1.100]
    maturity_targets = [30, 60, 90]
    cp_flags = ['C', 'P']

    # Preprocess base DataFrame
    spx_mod['days_to_maturity_int'] = spx_mod['days_to_maturity'].dt.days
    spx_mod = spx_mod.reset_index()
    spx_mod['original_index'] = spx_mod.index

    weight_results = []

    # Iterate through each strategy target
    for cp_flag in cp_flags:
        for target_moneyness in moneyness_targets:
            for target_ttm in maturity_targets:
                # Filter candidate options
                candidate_options = spx_mod[
                    (spx_mod['cp_flag'] == cp_flag) &
                    (spx_mod['moneyness_id'] == target_moneyness) &
                    (spx_mod['maturity_id'] == target_ttm)
                ].copy()

                if candidate_options.empty:
                    continue

                candidate_options['kernel_weight'] = np.nan

                # Apply kernel weights per date
                for date, g in candidate_options.groupby('date'):
                    idx = g.index
                    weights = kernel_weights(
                        g['moneyness'].values,
                        g['days_to_maturity_int'].values,
                        k_s=target_moneyness,
                        ttm=target_ttm
                    )
                    candidate_options.loc[idx, 'kernel_weight'] = weights

                weight_results.append(candidate_options[['original_index', 'kernel_weight']])

    # Merge weights back into spx_mod
    if weight_results:
        all_weights = pd.concat(weight_results).set_index('original_index')
        spx_mod.set_index('original_index', inplace=True)
        spx_mod['kernel_weight'] = all_weights['kernel_weight']
        spx_mod.reset_index(inplace=True)
    else:
        print("No matching options found for any target.")

    spx_mod.drop(columns=['original_index'], inplace=True)
    return spx_mod

In [7]:
def calc_option_delta_elasticity(df):
    """ Calculate option delta and elasticity for each option in the SPX dataset."""
    df = df\
        .assign(option_delta = lambda x: norm.cdf(
            (np.log(x['close'] / x['strike_price']) + (x['tb_m3'] + 0.5 * x['IV'] ** 2) * (x['days_to_maturity'].dt.days / 365.)) / (x['IV'] * np.sqrt(x['days_to_maturity'].dt.days / 365.))
            )
                ) \
        .assign(option_delta = lambda x: np.where(x['cp_flag'] == 'P', x['option_delta']-1, x['option_delta'])) \
        .assign(option_elasticity = lambda x: x['option_delta'] * x['close'] / x['mid_price'])
    return df

In [8]:
def read_option_data(filename):    
    # Example string interval: '(0.9, 0.95]'
    # Remove whitespace and parse the string into tuples
    def parse_interval_string(s):
        # Handle missing or malformed entries gracefully
        if pd.isnull(s) or not isinstance(s, str):
            return pd.NA  # or np.nan
        s = s.strip().replace('(', '').replace(']', '')
        try:
            left, right = map(float, s.split(','))
            return pd.Interval(left, right, closed='right')
        except ValueError:
            return pd.NA
    
    df = pd.read_parquet(filename)
    
    # restore the 'moneyness_bin' column as intervals
    df['moneyness_bin'] = df['moneyness_bin'].apply(parse_interval_string)
    
    return df

In [9]:
# read filtered data
source_file = Path(DATA_DIR / f'spx_filtered_final_{DATE_RANGE}.parquet')
spx_filtered = read_option_data(filename=source_file)
spx_filtered = spx_filtered.reset_index()

# create the moneyness ID from the moneyness_bin column, using the right edge of the interval
spx_filtered['moneyness_id'] = spx_filtered['moneyness_bin'].apply(lambda x: x.right if pd.notnull(x) else np.nan)
# drop any rows where moneyness_id is NaN
spx_filtered = spx_filtered.dropna(subset=['moneyness_id'])

spx_filtered

,date,exdate,moneyness,secid,open,close,cp_flag,IV,tb_m3,volume,open_interest,best_bid,best_offer,strike_price,contract_size,mid_price,days_to_maturity,pc_parity_int_rate,intrinsic,log_iv,fitted_iv,rel_distance_iv,moneyness_bin,stdev_iv_moneyness_bin,is_outlier_iv,moneyness_id
0,1996-01-04,1996-01-20,0.987534,108105.0,621.32,617.70,C,0.082711,5.04,444.0,5905.0,10.0000,10.375,610.0,100.0,10.18750,16 days,0.015898,7.70,-2.492403,-2.377298,4.841838,"(0.975, 1.0]",3.516840,False,1.000
1,1996-01-04,1996-01-20,1.019913,108105.0,621.32,617.70,C,0.097356,5.04,4022.0,5969.0,1.1875,1.375,630.0,100.0,1.28125,16 days,0.015898,0.00,-2.329381,-2.285771,1.907866,"(1.0, 1.025]",5.219336,False,1.025
2,1996-01-04,1996-01-20,1.028007,108105.0,621.32,617.70,C,0.101756,5.04,1627.0,6224.0,0.6250,0.750,635.0,100.0,0.68750,16 days,0.015898,0.00,-2.285177,-2.264082,0.931727,"(1.025, 1.05]",4.396845,False,1.050
3,1996-01-04,1996-01-20,1.036102,108105.0,621.32,617.70,C,0.100588,5.04,0.0,6593.0,0.1875,0.375,640.0,100.0,0.28125,16 days,0.015898,0.00,-2.296722,-2.242870,2.401027,"(1.025, 1.05]",4.396845,False,1.050
4,1996-01-04,1996-02-17,0.963251,108105.0,621.32,617.70,C,0.071852,5.04,3.0,34.0,25.2500,26.250,595.0,100.0,25.75000,44 days,0.014622,22.70,-2.633147,-2.563785,2.705450,"(0.95, 0.975]",2.301503,False,0.975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195881,2019-12-31,2020-06-19,1.044639,108105.0,3215.18,3230.78,P,0.117063,1.52,0.0,395.0,192.2000,193.000,3375.0,100.0,192.60000,171 days,0.014769,144.22,-2.145043,-2.132215,0.601630,"(1.025, 1.05]",4.396845,False,1.050
3195882,2019-12-31,2020-06-19,1.052377,108105.0,3215.18,3230.78,P,0.113900,1.52,0.0,163.0,208.2000,209.100,3400.0,100.0,208.65000,171 days,0.014769,169.22,-2.172434,-2.165558,0.317525,"(1.05, 1.075]",5.236635,False,1.075
3195883,2019-12-31,2020-06-19,1.060116,108105.0,3215.18,3230.78,P,0.110925,1.52,20.0,310.0,222.8000,228.900,3425.0,100.0,225.85000,171 days,0.014769,194.22,-2.198901,-2.199596,-0.031616,"(1.05, 1.075]",5.236635,False,1.075
3195884,2019-12-31,2020-06-19,1.067854,108105.0,3215.18,3230.78,P,0.108540,1.52,0.0,12.0,241.2000,247.600,3450.0,100.0,244.40000,171 days,0.014769,219.22,-2.220637,-2.234330,-0.612848,"(1.05, 1.075]",5.236635,False,1.075


## Leverage-Adjusted Portfolio Returns Construction Process

The construction of the 27 call and 27 put portfolios in CJS is a multi-step process, with the objective of developing portfolio returns series that are stationary and only moderately skewed. Note that the discrete bucketing of moneyness and days to maturity lead to multiple candidate options for each portfolio on each trading day. These options  are given weights according to a **bivariate Gaussian weighting kernel** in moneyness and maturity (bandwidths: *0.0125 in moneyness* and *10 days to maturity*).

Each portfolio's daily returns are initially calculated as simple arithmetic return, assuming the option is bought and sold at its bid-ask midpoint at each rebalancing. The one-day arithmetic return is then converted to a **leverage-adjusted return**. This procedure is achieved by calculating the one-day return of a hypothetical portfolio with $\omega_{BSM}^{-1}$ dollars invested in the option, and $(1 - \omega^{-1})$ dollars invested in the risk-free rate, where $\omega_{BSM}$ is the BSM elasticity based on the implied volatility of the option. 

\begin{align}
\omega_{\text{BSM, Call}} &= \frac{\partial C_{\text{BSM}}}{\partial S} \cdot \frac{S}{C_{\text{BSM}}} > 1 \\
\omega_{\text{BSM, Put}}  &= \frac{\partial P_{\text{BSM}}}{\partial S} \cdot \frac{S}{P_{\text{BSM}}} < -1
\end{align}

Each **leverage-adjusted call portfolio** comprises of a long position in a fraction of a call, and some investment in the risk-free rate. 

Each **leverage-adjusted put portfolio** comprises of a short position in a fraction of a put, and >100% investment in the risk-free rate. 



### 1. Build the FTSFA ID for each portfolio

In [10]:
# identify the maturity ID based on the closest maturity to 30, 60, or 90 days
maturity_id = pd.concat((abs(spx_filtered['days_to_maturity'].dt.days - 30), abs(spx_filtered['days_to_maturity'].dt.days - 60), abs(spx_filtered['days_to_maturity'].dt.days - 90)), axis=1)
maturity_id.columns = [30, 60, 90]
spx_filtered['maturity_id'] = maturity_id.idxmin(axis=1)
spx_filtered['ftfsa_id'] = spx_filtered['cp_flag'] + '_' + (spx_filtered['moneyness_id']*1000).apply(lambda x: str(int(x)) if pd.notnull(x) and x == int(x) else str(x)) \
    + '_' + spx_filtered['maturity_id'].astype(str)

# set index to ftfsa_id and date
spx_filtered.set_index(['ftfsa_id', 'date'], inplace=True)
spx_filtered

exdate  moneyness     secid     open    close  \
ftfsa_id  date                                                           
C_1000_30 1996-01-04 1996-01-20   0.987534  108105.0   621.32   617.70   
C_1025_30 1996-01-04 1996-01-20   1.019913  108105.0   621.32   617.70   
C_1050_30 1996-01-04 1996-01-20   1.028007  108105.0   621.32   617.70   
          1996-01-04 1996-01-20   1.036102  108105.0   621.32   617.70   
C_975_30  1996-01-04 1996-02-17   0.963251  108105.0   621.32   617.70   
...                         ...        ...       ...      ...      ...   
P_1050_90 2019-12-31 2020-06-19   1.044639  108105.0  3215.18  3230.78   
P_1075_90 2019-12-31 2020-06-19   1.052377  108105.0  3215.18  3230.78   
          2019-12-31 2020-06-19   1.060116  108105.0  3215.18  3230.78   
          2019-12-31 2020-06-19   1.067854  108105.0  3215.18  3230.78   
P_1100_90 2019-12-31 2020-06-19   1.075592  108105.0  3215.18  3230.78   

                     cp_flag        IV  tb_m3  volume  open_interest  \
ftfsa_id  date                                                         
C_1000_30 1996-01-04       C  0.082711   5.04   444.0         5905.0   
C_1025_30 1996-01-04       C  0.097356   5.04  4022.0         5969.0   
C_1050_30 1996-01-04       C  0.101756   5.04  1627.0         6224.0   
          1996-01-04       C  0.100588   5.04     0.0         6593.0   
C_975_30  1996-01-04       C  0.071852   5.04     3.0           34.0   
...                      ...       ...    ...     ...            ...   
P_1050_90 2019-12-31       P  0.117063   1.52     0.0          395.0   
P_1075_90 2019-12-31       P  0.113900   1.52     0.0          163.0   
          2019-12-31       P  0.110925   1.52    20.0          310.0   
          2019-12-31       P  0.108540   1.52     0.0           12.0   
P_1100_90 2019-12-31       P  0.106814   1.52     0.0            0.0   

                      best_bid  best_offer  strike_price  contract_size  \
ftfsa_id  date                                                            
C_1000_30 1996-01-04   10.0000      10.375         610.0          100.0   
C_1025_30 1996-01-04    1.1875       1.375         630.0          100.0   
C_1050_30 1996-01-04    0.6250       0.750         635.0          100.0   
          1996-01-04    0.1875       0.375         640.0          100.0   
C_975_30  1996-01-04   25.2500      26.250         595.0          100.0   
...                        ...         ...           ...            ...   
P_1050_90 2019-12-31  192.2000     193.000        3375.0          100.0   
P_1075_90 2019-12-31  208.2000     209.100        3400.0          100.0   
          2019-12-31  222.8000     228.900        3425.0          100.0   
          2019-12-31  241.2000     247.600        3450.0          100.0   
P_1100_90 2019-12-31  260.9000     267.500        3475.0          100.0   

                      mid_price days_to_maturity  pc_parity_int_rate  \
ftfsa_id  date                                                         
C_1000_30 1996-01-04   10.18750          16 days            0.015898   
C_1025_30 1996-01-04    1.28125          16 days            0.015898   
C_1050_30 1996-01-04    0.68750          16 days            0.015898   
          1996-01-04    0.28125          16 days            0.015898   
C_975_30  1996-01-04   25.75000          44 days            0.014622   
...                         ...              ...                 ...   
P_1050_90 2019-12-31  192.60000         171 days            0.014769   
P_1075_90 2019-12-31  208.65000         171 days            0.014769   
          2019-12-31  225.85000         171 days            0.014769   
          2019-12-31  244.40000         171 days            0.014769   
P_1100_90 2019-12-31  264.20000         171 days            0.014769   

                      intrinsic    log_iv  fitted_iv  rel_distance_iv  \
ftfsa_id  date                                                          
C_1000_30 1996-01-04       7.70 -2.492403  -2.377298         4.841

In [11]:
spx_filtered['days_to_maturity'].describe()

count                       2676696
mean     56 days 07:55:28.214186444
std      36 days 18:30:26.100585916
min                 7 days 00:00:00
25%                29 days 00:00:00
50%                49 days 00:00:00
75%                74 days 00:00:00
max               180 days 00:00:00
Name: days_to_maturity, dtype: object

### 2. Calculate option elasticity and daily kernel weighting for candidate options for each portfolio. 

In [12]:
spx_mod = spx_filtered.copy()

# calculate option delta and elasticity
spx_mod = calc_option_delta_elasticity(spx_mod)
# calculate daily kernel weights for candidate options
spx_mod = calc_kernel_weights(spx_mod)
spx_mod

,ftfsa_id,date,exdate,moneyness,secid,open,close,cp_flag,IV,tb_m3,volume,open_interest,best_bid,best_offer,strike_price,contract_size,mid_price,days_to_maturity,pc_parity_int_rate,intrinsic,log_iv,fitted_iv,rel_distance_iv,moneyness_bin,stdev_iv_moneyness_bin,is_outlier_iv,moneyness_id,maturity_id,option_delta,option_elasticity,days_to_maturity_int,kernel_weight
0,C_1000_30,1996-01-04,1996-01-20,0.987534,108105.0,621.32,617.70,C,0.082711,5.04,444.0,5905.0,10.0000,10.375,610.0,100.0,10.18750,16 days,0.015898,7.70,-2.492403,-2.377298,4.841838,"(0.975, 1.0]",3.516840,False,1.000,30,1.0,60.633129,16,4.123556e-01
1,C_1025_30,1996-01-04,1996-01-20,1.019913,108105.0,621.32,617.70,C,0.097356,5.04,4022.0,5969.0,1.1875,1.375,630.0,100.0,1.28125,16 days,0.015898,0.00,-2.329381,-2.285771,1.907866,"(1.0, 1.025]",5.219336,False,1.025,30,1.0,482.107317,16,1.000000e+00
2,C_1050_30,1996-01-04,1996-01-20,1.028007,108105.0,621.32,617.70,C,0.101756,5.04,1627.0,6224.0,0.6250,0.750,635.0,100.0,0.68750,16 days,0.015898,0.00,-2.285177,-2.264082,0.931727,"(1.025, 1.05]",4.396845,False,1.050,30,1.0,898.472727,16,2.829909e-01
3,C_1050_30,1996-01-04,1996-01-20,1.036102,108105.0,621.32,617.70,C,0.100588,5.04,0.0,6593.0,0.1875,0.375,640.0,100.0,0.28125,16 days,0.015898,0.00,-2.296722,-2.242870,2.401027,"(1.025, 1.05]",4.396845,False,1.050,30,1.0,2196.266667,16,7.170091e-01
4,C_975_30,1996-01-04,1996-02-17,0.963251,108105.0,621.32,617.70,C,0.071852,5.04,3.0,34.0,25.2500,26.250,595.0,100.0,25.75000,44 days,0.014622,22.70,-2.633147,-2.563785,2.705450,"(0.95, 0.975]",2.301503,False,0.975,30,1.0,23.988350,44,4.015525e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676691,P_1050_90,2019-12-31,2020-06-19,1.044639,108105.0,3215.18,3230.78,P,0.117063,1.52,0.0,395.0,192.2000,193.000,3375.0,100.0,192.60000,171 days,0.014769,144.22,-2.145043,-2.132215,0.601630,"(1.025, 1.05]",4.396845,False,1.050,90,0.0,0.000000,171,5.365533e-16
2676692,P_1075_90,2019-12-31,2020-06-19,1.052377,108105.0,3215.18,3230.78,P,0.113900,1.52,0.0,163.0,208.2000,209.100,3400.0,100.0,208.65000,171 days,0.014769,169.22,-2.172434,-2.165558,0.317525,"(1.05, 1.075]",5.236635,False,1.075,90,0.0,0.000000,171,1.775800e-16
2676693,P_1075_90,2019-12-31,2020-06-19,1.060116,108105.0,3215.18,3230.78,P,0.110925,1.52,20.0,310.0,222.8000,228.900,3425.0,100.0,225.85000,171 days,0.014769,194.22,-2.198901,-2.199596,-0.031616,"(1.05, 1.075]",5.236635,False,1.075,90,0.0,0.000000,171,4.495126e-16
2676694,P_1075_90,2019-12-31,2020-06-19,1.067854,108105.0,3215.18,3230.78,P,0.108540,1.52,0.0,12.0,241.2000,247.600,3450.0,100.0,244.40000,171 days,0.014769,219.22,-2.220637,-2.234330,-0.612848,"(1.05, 1.075]",5.236635,False,1.075,90,0.0,0.000000,171,7.756405e-16


In [13]:
print("Kernel weight check: Should sum to 1.0 for most portfolios on most days.")
spx_mod.groupby(['date', 'ftfsa_id'])['kernel_weight'].sum().round(15).value_counts()

Kernel weight check: Should sum to 1.0 for most portfolios on most days.


kernel_weight
1.0    140476
0.0     16046
Name: count, dtype: int64

***Note**: About 90% of the dataset has kernel weights that sum to 1.0, and 10% has kernel weights that sum to zero. Zero kernel weight for a given portfolio on a given date indicates that there were no candidate options on that particular date that were close to the moneyness and/or maturity targets, leading to all candidate options receiving low (or near-zero) weighting. This assertion needs to be investigated further.*

### 3. Remove options from the portfolio with weights lower than 1%

In [14]:
spx_mod = spx_mod[spx_mod['kernel_weight']>=0.01].reset_index(drop=True)
spx_mod

,ftfsa_id,date,exdate,moneyness,secid,open,close,cp_flag,IV,tb_m3,volume,open_interest,best_bid,best_offer,strike_price,contract_size,mid_price,days_to_maturity,pc_parity_int_rate,intrinsic,log_iv,fitted_iv,rel_distance_iv,moneyness_bin,stdev_iv_moneyness_bin,is_outlier_iv,moneyness_id,maturity_id,option_delta,option_elasticity,days_to_maturity_int,kernel_weight
0,C_1000_30,1996-01-04,1996-01-20,0.987534,108105.0,621.32,617.70,C,0.082711,5.04,444.0,5905.0,10.0000,10.375,610.0,100.0,10.18750,16 days,0.015898,7.70,-2.492403,-2.377298,4.841838,"(0.975, 1.0]",3.516840,False,1.000,30,1.000000e+00,6.063313e+01,16,0.412356
1,C_1025_30,1996-01-04,1996-01-20,1.019913,108105.0,621.32,617.70,C,0.097356,5.04,4022.0,5969.0,1.1875,1.375,630.0,100.0,1.28125,16 days,0.015898,0.00,-2.329381,-2.285771,1.907866,"(1.0, 1.025]",5.219336,False,1.025,30,1.000000e+00,4.821073e+02,16,1.000000
2,C_1050_30,1996-01-04,1996-01-20,1.028007,108105.0,621.32,617.70,C,0.101756,5.04,1627.0,6224.0,0.6250,0.750,635.0,100.0,0.68750,16 days,0.015898,0.00,-2.285177,-2.264082,0.931727,"(1.025, 1.05]",4.396845,False,1.050,30,1.000000e+00,8.984727e+02,16,0.282991
3,C_1050_30,1996-01-04,1996-01-20,1.036102,108105.0,621.32,617.70,C,0.100588,5.04,0.0,6593.0,0.1875,0.375,640.0,100.0,0.28125,16 days,0.015898,0.00,-2.296722,-2.242870,2.401027,"(1.025, 1.05]",4.396845,False,1.050,30,1.000000e+00,2.196267e+03,16,0.717009
4,C_975_30,1996-01-04,1996-02-17,0.963251,108105.0,621.32,617.70,C,0.071852,5.04,3.0,34.0,25.2500,26.250,595.0,100.0,25.75000,44 days,0.014622,22.70,-2.633147,-2.563785,2.705450,"(0.95, 0.975]",2.301503,False,0.975,30,1.000000e+00,2.398835e+01,44,0.401552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757525,P_1100_90,2019-12-31,2020-03-31,1.077139,108105.0,3215.18,3230.78,P,0.102622,1.52,0.0,0.0,251.8000,258.300,3480.0,100.0,255.05000,91 days,0.015048,249.22,-2.276703,-2.342683,-2.816445,"(1.075, 1.1]",5.723928,False,1.100,90,-1.178312e-09,-1.492597e-08,91,0.066978
1757526,P_1100_90,2019-12-31,2020-03-31,1.080234,108105.0,3215.18,3230.78,P,0.099399,1.52,0.0,0.0,258.2000,269.200,3490.0,100.0,263.70000,91 days,0.015048,259.22,-2.308613,-2.356986,-2.052300,"(1.075, 1.1]",5.723928,False,1.100,90,-5.131693e-10,-6.287209e-09,91,0.102163
1757527,P_1100_90,2019-12-31,2020-03-31,1.083330,108105.0,3215.18,3230.78,P,0.102206,1.52,0.0,2.0,267.7000,279.600,3500.0,100.0,273.65000,91 days,0.015048,269.22,-2.280765,-2.371357,-3.820268,"(1.075, 1.1]",5.723928,False,1.100,90,-2.015702e-09,-2.379788e-08,91,0.146563
1757528,P_1100_90,2019-12-31,2020-03-31,1.091068,108105.0,3215.18,3230.78,P,0.103736,1.52,0.0,0.0,291.6000,303.600,3525.0,100.0,297.60000,91 days,0.015048,294.22,-2.265906,-2.407588,-5.884824,"(1.075, 1.1]",5.723928,False,1.100,90,-7.591612e-09,-8.241542e-08,91,0.276281


In [15]:
# check elasticity > 1 for call options and < -1 for put options
spx_mod[spx_mod['cp_flag']=='C']['option_elasticity'].describe()

count    871163.000000
mean        328.217618
std        1590.030529
min           3.855476
25%          14.944374
50%          29.327019
75%          80.377220
max       40087.292970
Name: option_elasticity, dtype: float64

In [16]:
spx_mod[spx_mod['cp_flag']=='P']['option_elasticity'].describe()

count    871163.000000
mean        -12.883676
std          11.170971
min        -144.674609
25%         -18.491388
50%         -11.583938
75%          -3.506240
max           0.000000
Name: option_elasticity, dtype: float64

***Note**: Put elasticity needs to be < -1, but we have several observations > -1 (and none > 0). This is to be investigated further.*

### 4. Calculate the daily arithmetic return of each portfolio (which comprises of a weighted set of candidate options).

On each trading day, the return of a portfolio is calculated as weighted average return of the set of candidate options that comprise a single day's option portfolio. The weighting used is the Gaussian kernel weight calculated earlier. Thus the daily return from period $t$ to $t+1$ represents the return from holding a set of candidate options, weighted using the kernel weights as of $t$, from period $t$ to $t+1$. 

In [17]:
## NEXT CALCULATE RETURNS

XXX

In [18]:
spx_mod

,ftfsa_id,date,exdate,moneyness,secid,open,close,cp_flag,IV,tb_m3,volume,open_interest,best_bid,best_offer,strike_price,contract_size,mid_price,days_to_maturity,pc_parity_int_rate,intrinsic,log_iv,fitted_iv,rel_distance_iv,moneyness_bin,stdev_iv_moneyness_bin,is_outlier_iv,moneyness_id,maturity_id,option_delta,option_elasticity,days_to_maturity_int,kernel_weight
0,C_1000_30,1996-01-04,1996-01-20,0.987534,108105.0,621.32,617.70,C,0.082711,5.04,444.0,5905.0,10.0000,10.375,610.0,100.0,10.18750,16 days,0.015898,7.70,-2.492403,-2.377298,4.841838,"(0.975, 1.0]",3.516840,False,1.000,30,1.000000e+00,6.063313e+01,16,0.412356
1,C_1025_30,1996-01-04,1996-01-20,1.019913,108105.0,621.32,617.70,C,0.097356,5.04,4022.0,5969.0,1.1875,1.375,630.0,100.0,1.28125,16 days,0.015898,0.00,-2.329381,-2.285771,1.907866,"(1.0, 1.025]",5.219336,False,1.025,30,1.000000e+00,4.821073e+02,16,1.000000
2,C_1050_30,1996-01-04,1996-01-20,1.028007,108105.0,621.32,617.70,C,0.101756,5.04,1627.0,6224.0,0.6250,0.750,635.0,100.0,0.68750,16 days,0.015898,0.00,-2.285177,-2.264082,0.931727,"(1.025, 1.05]",4.396845,False,1.050,30,1.000000e+00,8.984727e+02,16,0.282991
3,C_1050_30,1996-01-04,1996-01-20,1.036102,108105.0,621.32,617.70,C,0.100588,5.04,0.0,6593.0,0.1875,0.375,640.0,100.0,0.28125,16 days,0.015898,0.00,-2.296722,-2.242870,2.401027,"(1.025, 1.05]",4.396845,False,1.050,30,1.000000e+00,2.196267e+03,16,0.717009
4,C_975_30,1996-01-04,1996-02-17,0.963251,108105.0,621.32,617.70,C,0.071852,5.04,3.0,34.0,25.2500,26.250,595.0,100.0,25.75000,44 days,0.014622,22.70,-2.633147,-2.563785,2.705450,"(0.95, 0.975]",2.301503,False,0.975,30,1.000000e+00,2.398835e+01,44,0.401552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757525,P_1100_90,2019-12-31,2020-03-31,1.077139,108105.0,3215.18,3230.78,P,0.102622,1.52,0.0,0.0,251.8000,258.300,3480.0,100.0,255.05000,91 days,0.015048,249.22,-2.276703,-2.342683,-2.816445,"(1.075, 1.1]",5.723928,False,1.100,90,-1.178312e-09,-1.492597e-08,91,0.066978
1757526,P_1100_90,2019-12-31,2020-03-31,1.080234,108105.0,3215.18,3230.78,P,0.099399,1.52,0.0,0.0,258.2000,269.200,3490.0,100.0,263.70000,91 days,0.015048,259.22,-2.308613,-2.356986,-2.052300,"(1.075, 1.1]",5.723928,False,1.100,90,-5.131693e-10,-6.287209e-09,91,0.102163
1757527,P_1100_90,2019-12-31,2020-03-31,1.083330,108105.0,3215.18,3230.78,P,0.102206,1.52,0.0,2.0,267.7000,279.600,3500.0,100.0,273.65000,91 days,0.015048,269.22,-2.280765,-2.371357,-3.820268,"(1.075, 1.1]",5.723928,False,1.100,90,-2.015702e-09,-2.379788e-08,91,0.146563
1757528,P_1100_90,2019-12-31,2020-03-31,1.091068,108105.0,3215.18,3230.78,P,0.103736,1.52,0.0,0.0,291.6000,303.600,3525.0,100.0,297.60000,91 days,0.015048,294.22,-2.265906,-2.407588,-5.884824,"(1.075, 1.1]",5.723928,False,1.100,90,-7.591612e-09,-8.241542e-08,91,0.276281


In [19]:
# make a 3-d line+marker plot of the 2019 call option delta over time. x-axis is date, y-axis is option delta, and z-axis is moneyness
fig = px.line_3d(
    spx_mod.loc[pd.IndexSlice['01-01-2018':'03-01-2018', :, :, :]].reset_index(),
    x='date', y='moneyness', z='option_delta',
    markers=True
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(
    scene=dict(
        xaxis_title='Date',
        yaxis_title='Moneyness',
        zaxis_title='Option Delta'
    ),
    width=800,
    height=1200
)
fig.show()

IndexingError: Too many indexers

In [ ]:
# calc option deltas

d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
if option_type == 'call':
    price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T))
    delta = norm.cdf(d1)
else:
    price = K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)) - S * norm.cdf(-d1)
    delta = -norm.cdf(-d1)

In [ ]:
pd.date_tDATE_RANGE

In [ ]:
moneyness_tgts = [0.90, 0.925, 0.95, 0.975, 1.00, 1.025, 1.05, 1.075, 1.10] # K/S, so < 1 = puts are OTM/calls are ITM, and > 1 calls are OTM/puts are ITM
maturity_tgts = [30, 60, 90]  # in days. For HKM, these are averaged
option_types = ['C', 'P']  # 'call' and 'put' options

portfolio_idx = pd.MultiIndex.from_tuples(
    [(opt_type, k_s, ttm) for opt_type in option_types for k_s in moneyness_tgts for ttm in maturity_tgts for trade_date in spx_filtered.index.get_level_values('date')],
    names=['option_type', 'moneyness', 'days_to_maturity', 'trade_date']
)

portfolios = pd.DataFrame(index=portfolio_idx, columns=['daily_rets', 'monthly_rets'])
portfolios